In [1]:
import pandas as pd
from PIL import Image
from skimage import color

import numpy as np
# from colour import XYZ_to_CAM16
import numpy as np
# from colour.models import RGB_to_XYZ
# 读取图片
img = Image.open('image/101393174_p0.png')

# 获取图片尺寸
width, height = img.size

# 获取像素数据
pixels = list(img.getdata())

# 创建DataFrame,A代表透明度
try:
    df = pd.DataFrame(pixels, columns=['R', 'G', 'B','A'])
    df = df.drop(columns=['A'])
except:
    df = pd.DataFrame(pixels, columns=['R', 'G', 'B'])
# 添加X和Y列
df['X'] = df.index % width
df['Y'] = df.index // width

# 显示DataFrame
print(df)

         R   G   B    X     Y
0        0   3   4    0     0
1        0   3   4    1     0
2        0   3   4    2     0
3        0   3   4    3     0
4        0   3   4    4     0
...     ..  ..  ..  ...   ...
1046657  1  31  31  833  1248
1046658  0  28  26  834  1248
1046659  0  25  24  835  1248
1046660  0  20  21  836  1248
1046661  0  16  15  837  1248

[1046662 rows x 5 columns]


In [2]:
from colorspacious import cspace_converter
RGB = np.array([df['R'], df['G'], df['B']]).T

converter = cspace_converter("sRGB255", "CIELCh")
# 将LCH颜色添加到DataFrame
lch = converter(RGB)
df['L'] = lch[:, 0]
df['C'] = lch[:, 1]
df['H'] = lch[:, 2]
df['L'] = df['L'].round()
df['C'] = df['C'].round()
df['H'] = df['H'].round()
df['count'] = 1
df


,R,G,B,X,Y,L,C,H,count
0,0,3,4,0,0,1.0,1.0,225.0,1
1,0,3,4,1,0,1.0,1.0,225.0,1
2,0,3,4,2,0,1.0,1.0,225.0,1
3,0,3,4,3,0,1.0,1.0,225.0,1
4,0,3,4,4,0,1.0,1.0,225.0,1
...,...,...,...,...,...,...,...,...,...
1046657,1,31,31,833,1248,10.0,12.0,197.0,1
1046658,0,28,26,834,1248,8.0,11.0,190.0,1
1046659,0,25,24,835,1248,7.0,9.0,194.0,1
1046660,0,20,21,836,1248,5.0,6.0,205.0,1


In [3]:
C_mean = df['C'].mean()
L_mean = df['L'].mean()
H_mean = df['H'].mean()

In [4]:
df_mean = df.groupby('H').agg({'R': 'mean', 'G': 'mean', 'B': 'mean','L': 'mean','C':'mean'}).reset_index()
df_mean['R'] = df_mean['R'].round().astype(int)
df_mean['G'] = df_mean['G'].round().astype(int)
df_mean['B'] = df_mean['B'].round().astype(int)
df_mean['L'] = df_mean['L'].round().astype(int)
df_mean['C'] = df_mean['C'].round().astype(int)
df_count = df.groupby([ 'H']).size().reset_index(name='v')
df_merged = pd.merge(df_mean, df_count, on='H')
df_merged

,H,R,G,B,L,C,v
0,0.0,50,44,46,18,3,93
1,1.0,123,106,110,44,7,35
2,2.0,131,105,112,46,12,18
3,3.0,181,161,165,67,9,34
4,4.0,166,146,150,62,8,117
...,...,...,...,...,...,...,...
356,356.0,104,86,92,37,8,44
357,357.0,36,23,27,9,7,25
358,358.0,34,15,22,7,8,7
359,359.0,193,188,189,76,2,121


In [5]:
import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('H')

colors = ['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_merged_sorted['R'], df_merged_sorted['G'], df_merged_sorted['B'])]


fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图（按色相顺序）', showlegend=False)
fig.show()

In [6]:
fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none'))
fig.update_layout(title='色相分布图（按每个色相份额排序）', showlegend=False)
fig.show()

In [7]:
import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('L')

colors = ['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_merged_sorted['R'], df_merged_sorted['G'], df_merged_sorted['B'])]


fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图（按亮度排序）', showlegend=False)
fig.show()

In [8]:




import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('H')

# Calculate LCH values based on C_mean, L_mean, and H
lch_colors = [(L_mean, C_mean, h) for h in df_merged_sorted['H']]

# Convert LCH colors to RGB
converter = cspace_converter("CIELCh", "sRGB255")
rgb_colors = converter(lch_colors)

# Format RGB colors
colors = ['rgb({},{},{})'.format(int(r), int(g), int(b)) for r, g, b in rgb_colors]

fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图(平均色度和亮度后)', showlegend=False)
fig.show()

In [9]:
df_learn = df.groupby(['C', 'H']).agg({ 'L': 'sum'}).reset_index()
df_learn.columns = ['C', 'H', 'L']
df_learn['C_36']= df_learn['C']*3.6

In [10]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=256, init='k-means++',algorithm='lloyd')
kmeans.fit(df_learn[['C_36', 'H',]], sample_weight=df_learn['L'])
# 获取聚类结果
df_predict = df.copy()
df_predict['C_36'] = df_predict['C']*3.6
labels = kmeans.predict(df_predict[['C_36', 'H']])
# 将聚类结果写回原始数据框中

In [11]:
df['Cluster'] = labels
df

,R,G,B,X,Y,L,C,H,count,Cluster
0,0,3,4,0,0,1.0,1.0,225.0,1,33
1,0,3,4,1,0,1.0,1.0,225.0,1,33
2,0,3,4,2,0,1.0,1.0,225.0,1,33
3,0,3,4,3,0,1.0,1.0,225.0,1,33
4,0,3,4,4,0,1.0,1.0,225.0,1,33
...,...,...,...,...,...,...,...,...,...,...
1046657,1,31,31,833,1248,10.0,12.0,197.0,1,207
1046658,0,28,26,834,1248,8.0,11.0,190.0,1,170
1046659,0,25,24,835,1248,7.0,9.0,194.0,1,156
1046660,0,20,21,836,1248,5.0,6.0,205.0,1,70


In [12]:
df_learn_show = df.groupby(['Cluster']).agg({ 'L': 'sum','C': 'mean','H': 'mean','R': 'mean', 'G': 'mean', 'B': 'mean','count': 'sum'}).reset_index()
df_learn_show['C'] = df_learn_show['C'].round().astype(int)
df_learn_show['H'] = df_learn_show['H'].round().astype(int)
df_learn_show['R'] = df_learn_show['R'].round().astype(int)
df_learn_show['G'] = df_learn_show['G'].round().astype(int)
df_learn_show['B'] = df_learn_show['B'].round().astype(int)

df_learn_show

,Cluster,L,C,H,R,G,B,count
0,0,127059.0,3,89,129,127,122,2448
1,1,204971.0,11,213,4,34,39,18129
2,2,91368.0,32,31,162,93,87,1959
3,3,57875.0,57,70,242,167,84,779
4,4,133847.0,22,87,181,162,125,2004
...,...,...,...,...,...,...,...,...
251,251,66288.0,59,56,233,135,73,1006
252,252,59231.0,3,39,125,119,117,1207
253,253,200872.0,9,204,15,38,39,15509
254,254,212663.0,14,204,7,50,52,11719


In [13]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_show['C'],
    y=df_learn_show['H'],
    mode='markers',
    marker=dict(
        size=np.sqrt(df_learn_show['count'])/8,
        color=df_learn_show.apply(lambda row: f'rgb({row["R"]},{row["G"]},{row["B"]})', axis=1),
        opacity=0.8
    ),
    text=df_learn_show.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='对色彩进行聚类后的结果',
    xaxis=dict(title='C'),
    yaxis=dict(title='H'),
    showlegend=False
)

fig.show()


In [14]:
from sklearn.cluster import KMeans
kmeans2 = KMeans(n_clusters=5, init='k-means++',algorithm='lloyd')
kmeans2.fit(df_learn_show[['H','C']],sample_weight=df_learn_show['L'])
# 获取聚类结果
labels2 = kmeans2.predict(df_learn_show[[ 'H','C']])
df_learn_show['Cluster2'] = labels2
df_learn_show
# 将聚类结果写回原始数据框中 取出df_learn_show中cluster2相同的行中L值最大的行，用plotly绘制饼图，颜色用R，G，B值，大小用count值

,Cluster,L,C,H,R,G,B,count,Cluster2
0,0,127059.0,3,89,129,127,122,2448,1
1,1,204971.0,11,213,4,34,39,18129,0
2,2,91368.0,32,31,162,93,87,1959,2
3,3,57875.0,57,70,242,167,84,779,2
4,4,133847.0,22,87,181,162,125,2004,1
...,...,...,...,...,...,...,...,...,...
251,251,66288.0,59,56,233,135,73,1006,2
252,252,59231.0,3,39,125,119,117,1207,2
253,253,200872.0,9,204,15,38,39,15509,0
254,254,212663.0,14,204,7,50,52,11719,0


In [15]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_show['C'],
    y=df_learn_show['H'],
    mode='markers',
    marker=dict(
        size=np.sqrt(df_learn_show['count'])/8,
        color=df_learn_show['Cluster2'],
        opacity=0.8
    ),
    text=df_learn_show.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='对色相聚一次类',
    xaxis=dict(title='C：色度'),
    yaxis=dict(title='H：色相'),
    showlegend=False
)

fig.show()

In [16]:
df_learn_show['final_v']=df_learn_show['C']*df_learn_show['L']

In [17]:
df_learn_l = df.groupby(['L']).agg({ 'count': 'sum'}).reset_index()

In [18]:
df_learn_show['L'] = (df_learn_show['L']/df_learn_show['count']).round().astype(int)

In [19]:
from sklearn.cluster import KMeans
kmeans3 = KMeans(n_clusters=10, init='k-means++',algorithm='lloyd')
kmeans3.fit(df_learn_l[['L']],sample_weight=df_learn_l['count'])
# 获取聚类结果
labels3 = kmeans3.predict(df_learn_l[[ 'L']])
df_learn_l['Cluster3'] = labels3
df_learn_l
# 将聚类结果写回原始数据框中 取出df_learn_show中cluster2相同的行中L值最大的行，用plotly绘制饼图，颜色用R，G，B值，大小用count值

,L,count,Cluster3
0,0.0,3602,0
1,1.0,127601,0
2,2.0,96764,0
3,3.0,66049,0
4,4.0,48442,0
...,...,...,...
96,96.0,1029,3
97,97.0,949,3
98,98.0,1319,3
99,99.0,611,3


In [20]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_l['L'],
    y=df_learn_l['count'],
    mode='markers',
    marker=dict(
        color=df_learn_l['Cluster3'],
        colorscale='Viridis',
        opacity=0.8
    )
))

fig.update_layout(
    title='在亮度上对像素聚类',
    xaxis=dict(title='L：亮度'),
    yaxis=dict(title='像素量'),
    showlegend=False
)

fig.show()


In [21]:
df_learn_l

,L,count,Cluster3
0,0.0,3602,0
1,1.0,127601,0
2,2.0,96764,0
3,3.0,66049,0
4,4.0,48442,0
...,...,...,...
96,96.0,1029,3
97,97.0,949,3
98,98.0,1319,3
99,99.0,611,3


In [22]:
df_l_plan = df_learn_l.groupby('Cluster3').apply(lambda x: x.loc[x['count'].idxmax()]).reset_index(drop=True)
df_l_plan = df_l_plan.apply(lambda x: x.sort_values().reset_index(drop=True))
df_l_plan

/tmp/ipykernel_157548/210686816.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,L,count,Cluster3
0,1.0,3964.0,0.0
1,5.0,4728.0,1.0
2,11.0,5067.0,2.0
3,25.0,5261.0,3.0
4,29.0,5279.0,4.0
5,48.0,6557.0,5.0
6,58.0,13538.0,6.0
7,69.0,21777.0,7.0
8,71.0,37917.0,8.0
9,86.0,127601.0,9.0


In [23]:
df_color_plan = df_learn_show.groupby('Cluster2').apply(lambda x: x.loc[x['final_v'].idxmax()]).reset_index(drop=True)
df_color_plan

/tmp/ipykernel_157548/2810096399.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,Cluster,L,C,H,R,G,B,count,Cluster2,final_v
0,60.0,16.0,14.0,194.0,7.0,46.0,45.0,21150.0,0.0,4794020.0
1,132.0,69.0,25.0,87.0,188.0,166.0,123.0,2115.0,1.0,3635075.0
2,47.0,48.0,69.0,37.0,206.0,62.0,46.0,1933.0,2.0,6426936.0
3,11.0,50.0,40.0,118.0,106.0,126.0,57.0,889.0,3.0,1762720.0
4,184.0,64.0,39.0,241.0,15.0,166.0,213.0,364.0,4.0,902031.0


In [24]:
df_plan = pd.DataFrame(columns=['L','C','H','R','G','B','count','count2','cluster2','cluster3'])

converter2 = cspace_converter("CIELCh","sRGB255")
# 将LCH颜色添加到DataFrame

lch = converter(RGB)
for idx, data in df_color_plan.iterrows():
    for idx2, data2 in df_l_plan.iterrows():
        L = data2['L']
        C = data['C']
        H = data['H']
        RGB = converter2((L, C, H))
        R = RGB[0]
        G = RGB[1]
        B = RGB[2]
        count = data['count']
        count2 = data2['count']
        cluster2 = data['Cluster2']
        cluster3 = data2['Cluster3']
        df_plan.loc[len(df_plan)] = [L,C,H,R,G,B,count,count2,cluster2,cluster3]

df_plan = df_plan.applymap(lambda x: 0 if x < 0 else x)
df_plan


/tmp/ipykernel_157548/1174046150.py:22: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,L,C,H,R,G,B,count,count2,cluster2,cluster3
0,1.0,14.0,194.0,0.000000,14.071508,11.242897,21150.0,3964.0,0.0,0.0
1,5.0,14.0,194.0,0.000000,23.497336,21.678681,21150.0,4728.0,0.0,1.0
2,11.0,14.0,194.0,0.000000,34.446905,33.758744,21150.0,5067.0,0.0,2.0
3,25.0,14.0,194.0,27.873965,65.169603,64.024869,21150.0,5261.0,0.0,3.0
4,29.0,14.0,194.0,37.399961,74.421700,73.178359,21150.0,5279.0,0.0,4.0
5,48.0,14.0,194.0,83.033920,120.769423,119.162677,21150.0,6557.0,0.0,5.0
6,58.0,14.0,194.0,108.085354,146.557953,144.805050,21150.0,13538.0,0.0,6.0
7,69.0,14.0,194.0,136.565194,175.891874,174.000845,21150.0,21777.0,0.0,7.0
8,71.0,14.0,194.0,141.844449,181.326769,179.412663,21150.0,37917.0,0.0,8.0
9,86.0,14.0,194.0,182.380685,223.010897,220.939482,21150.0,127601.0,0.0,9.0


In [25]:
df_color_plan

,Cluster,L,C,H,R,G,B,count,Cluster2,final_v
0,60.0,16.0,14.0,194.0,7.0,46.0,45.0,21150.0,0.0,4794020.0
1,132.0,69.0,25.0,87.0,188.0,166.0,123.0,2115.0,1.0,3635075.0
2,47.0,48.0,69.0,37.0,206.0,62.0,46.0,1933.0,2.0,6426936.0
3,11.0,50.0,40.0,118.0,106.0,126.0,57.0,889.0,3.0,1762720.0
4,184.0,64.0,39.0,241.0,15.0,166.0,213.0,364.0,4.0,902031.0


In [26]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_plan['cluster3'],
    y=df_plan['cluster2'],
    mode='markers',
    marker=dict(
        size=32,
        color=df_plan.apply(lambda row: f'rgb({row["R"]},{row["G"]},{row["B"]})', axis=1),
        opacity=0.8
    ),
    text=df_plan.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='配色方案',
    xaxis=dict(title='亮度'),
    yaxis=dict(title='颜色'),
    showlegend=False
)

fig.show()

In [27]:
import plotly.graph_objects as go

# Group the DataFrame by 'Cluster2' and select the row with the maximum 'L' value
df_max_l = df_learn_show.groupby('Cluster2').apply(lambda x: x.loc[x['final_v'].idxmax()]).reset_index(drop=True)

# Create the pie chart
fig = go.Figure(data=go.Pie(
    labels=df_max_l['Cluster2'],
    values=df_max_l['count'],
    marker=dict(
        colors=['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_max_l['R'], df_max_l['G'], df_max_l['B'])]
    ),
    textinfo='none'
))

fig.update_layout(title='取色结果（如果图片颜色太少区别会很小，这不是bug是feature😡）', showlegend=False)
fig.show()


/tmp/ipykernel_157548/1545999088.py:4: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

